# Coursework 3: Reed-Solomon Codes

[] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

**Julia Packages**

We use the `LinearAlgebra` package for matrix operations.
We use the `StatsBase` package for introducing random error.

In [1]:
using Pkg
Pkg.add("LinearAlgebra")
Pkg.add("StatsBase")

using LinearAlgebra
using StatsBase

    Updating registry at `C:\Users\aydes\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\aydes\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\aydes\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\aydes\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\aydes\.julia\environments\v1.6\Manifest.toml`


#### Helper Functions

In [2]:
# Helper Functions

# Function to do modulo division
function modulo_div(num,den,p)
    return mod((num * gcdx(den,p)[2]),p)
end

# Function for modular exoponentiation
function modExpo(p,x,n)
    if p == 1 
        return 0
    end
    a = 1 # initialise a
    binaryExp = mod(x,p) #binaryExp when i=0
    while n > 0 #loop cycles through each binary digit of n - (cycle of ceiling[log2(n)] times)
        if (mod(n,2) == 1) #check LSB
            a = mod((a * binaryExp), p) #if LSB is 1 then update a accordingly
        end
        n = n >> 1 #right shift exponent to check next bit
        binaryExp = mod((binaryExp * binaryExp), p) #calculate new binaryExp value by squaring previous result and taking modulus
    end
    return a
end

modExpo (generic function with 1 method)

In [42]:
struct MyPolynomial{T}
    coeffs::Vector{T}
    order::Int
    p::Int
end

function ConstructMyPolynomial(coeffs,p)
    coeffs = [mod(i,p) for i in coeffs]
    return MyPolynomial(coeffs,length(coeffs) - 1,p)
end

function remove_top_zeros(poly::MyPolynomial)
    p = poly.p
    coeffs = poly.coeffs
    if typeof(poly) == MyPolynomial{Int}
        zero_term = 0
    else
        zero_term = ConstructMyPolynomial([0],p)
    end
    while length(coeffs) > 0 && last(coeffs) == zero_term
        pop!(coeffs)
    end
    if length(coeffs) == 0
        push!(coeffs,zero_term)
    end
    return ConstructMyPolynomial(coeffs,poly.p)
end

function make_same_length(poly1::MyPolynomial{Int}, poly2::MyPolynomial{Int})
    len1 = length(poly1.coeffs)
    len2 = length(poly2.coeffs)

    if len1 > len2
        zero_pad = zeros(Int,len1-len2)
        new_coeffs = cat(poly2.coeffs,zero_pad,dims=1)
        new_poly = ConstructMyPolynomial(new_coeffs,poly2.p)
        return (poly1,new_poly)
    else
        zero_pad = zeros(Int,len2-len1)
        new_coeffs = cat(poly1.coeffs,zero_pad,dims=1)
        new_poly = ConstructMyPolynomial(new_coeffs,poly1.p)
        return (poly2,new_poly)

    end
end

function make_same_length(poly1::MyPolynomial{MyPolynomial{Int}}, poly2::MyPolynomial{MyPolynomial{Int}})
    p = poly1.p
    len1 = length(poly1.coeffs)
    len2 = length(poly2.coeffs)

    if len1 > len2
        zero_pad = zeros_poly(len1-len2,p)
        new_coeffs = cat(poly2.coeffs,zero_pad,dims=1)
        new_poly = ConstructMyPolynomial(new_coeffs,poly2.p)
        return (poly1,new_poly)
    else 
        zero_pad = zeros_poly(len2-len1,p)
        new_coeffs = cat(poly1.coeffs,zero_pad,dims=1)
        new_poly = ConstructMyPolynomial(new_coeffs,poly1.p)
        return (new_poly,poly2)
    end
end

function Base.:mod(poly::MyPolynomial,p::Int)
    new_coeffs = [mod(i,p) for i in poly.coeffs]
    return ConstructMyPolynomial(new_coeffs,p)
end

function Base.:mod(poly1::MyPolynomial,poly2::MyPolynomial)
    return divrem(poly1,poly2)[2]
end

function Base.:length(poly::MyPolynomial)
    return length(poly.coeffs)
end

function zeros_poly(num::Int, p::Int)
    zero_poly = ConstructMyPolynomial([0],p)
    z = []
    for i in 1:num
        push!(z,zero_poly)
    end
    return z
end

function ones_poly(num::Int, p::Int)
    one_poly = ConstructMyPolynomial([1],p)
    z = []
    for i in 1:num
        push!(z,one_poly)
    end
    return z
end

function Base.:fill(poly::MyPolynomial, num::Int)
    poly = remove_top_zeros(poly)
    z = []
    for i in 1:num
        push!(z,poly)
    end
    return z
end

function Base.:+(poly1::MyPolynomial, poly2::MyPolynomial)
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    poly1,poly2 = make_same_length(poly1,poly2)

    new_coeffs = poly1.coeffs + poly2.coeffs
    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end

function Base.:*(poly::MyPolynomial, num::Int)
    p = poly.p
    new_coeffs = [mod(i,p) for i in (poly.coeffs * num)]
    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end

function Base.:*(num::Int, poly::MyPolynomial)
    return poly * num
end

function Base.:-(poly1::MyPolynomial, poly2::MyPolynomial)
    return poly1 + (-1 * poly2)
end

function Base.:*(poly1::MyPolynomial{Int}, poly2::MyPolynomial{Int})
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    c1 = poly1.coeffs
    c2 = poly2.coeffs
    m = length(c1)
    n = length(c2)

    new_coeffs = zeros(Int,m+n-1)
    for i in 1:m
        for j in 1:n
            new_coeffs[i+j-1] += c1[i]*c2[j]
        end
    end

    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end

function Base.:*(poly1::MyPolynomial{MyPolynomial{Int}}, poly2::MyPolynomial{MyPolynomial{Int}})
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    c1 = poly1.coeffs
    c2 = poly2.coeffs
    m = length(c1)
    n = length(c2)

    new_coeffs = zeros_poly(m+n-1,p)
    for i in 1:m
        for j in 1:n
            new_coeffs[i+j-1] += c1[i]*c2[j]
        end
    end

    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end

function Base.:^(poly::MyPolynomial, num::Int)
    if num == 0
        return ConstructMyPolynomial([1],poly.p)
    end
    
    poly_acc = ConstructMyPolynomial(poly.coeffs,poly.p)
    for i in 2:num
        poly_acc = poly_acc * poly
    end
    return poly_acc
end

function Base.:(==)(poly1::MyPolynomial,poly2::MyPolynomial)
    return poly1.coeffs == poly2.coeffs
end

function Base.:(!=)(poly1::MyPolynomial,poly2::MyPolynomial)
    return poly1.coeffs != poly2.coeffs
end

function Base.:zero(x::MyPolynomial{Int})
    return ConstructMyPolynomial([0],x.p)
end

function Base.:one(x::MyPolynomial{Int})
    return ConstructMyPolynomial([1],x.p)
end

function Base.:divrem(poly1::MyPolynomial{Int}, poly2::MyPolynomial{Int})
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    num = poly1.coeffs
    den = poly2.coeffs

    if length(num) >= length(den)
        # Shift denominator to the right to match degree of numerator.
        shiftamount = length(num) - length(den)
        den = append!(zeros(Int,shiftamount),den)
    else
        # This is the case where degree of denominator > degree of numerator.
        # Results in zero quotient and the numerator being the remainder.
        return (ConstructMyPolynomial([0],p),remove_top_zeros(ConstructMyPolynomial(num,p)),)
    end

    quotient = []
    divisor = last(den)

    for i in 1:(shiftamount+1)
        #Get the next coefficient of the quotient.
        mult = modulo_div(last(num),divisor,p)
        quotient = append!([mult],quotient)
        
        if mult != 0
            d = [mult * u for u in den]
            for i in 1:length(num)
                num[i] = mod((num[i] - d[i]),p)
            end
        end
        
        pop!(num)
        popfirst!(den)
    end

    return (remove_top_zeros(ConstructMyPolynomial(quotient,p)),remove_top_zeros(ConstructMyPolynomial(num,p)))

end

function Base.:divrem(poly1::MyPolynomial{MyPolynomial{Int}}, poly2::MyPolynomial{MyPolynomial{Int}})
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    num = poly1.coeffs
    den = poly2.coeffs

    if length(num) >= length(den)
        # Shift denominator to the right to match degree of numerator.
        shiftamount = length(num) - length(den)
        den = append!(zeros_poly(shiftamount,p),den)
    else
        # This is the case where degree of denominator > degree of numerator.
        # Results in zero quotient and the numerator being the remainder.
        return (ConstructMyPolynomial([0],p),remove_top_zeros(ConstructMyPolynomial(num,p)))
    end

    quotient = []
    divisor = last(den)

    for i in 1:(shiftamount+1)
        #Get the next coefficient of the quotient.
        mult = divrem(last(num),divisor)[1]
        quotient = append!([mult],quotient)
        
        if mult != 0
            d = [mult * u for u in den]
            for i in 1:length(num)
                num[i] = mod((num[i] - d[i]),p)
            end
        end
        
        pop!(num)
        popfirst!(den)
    end

    return (remove_top_zeros(ConstructMyPolynomial(quotient,p)),remove_top_zeros(ConstructMyPolynomial(num,p)),)

end

function Base.:powermod(poly::MyPolynomial, num::Int, f::MyPolynomial)
    if num == 0
        return ConstructMyPolynomial([1],poly.p)
    end
    f = remove_top_zeros(f)
    poly_acc = remove_top_zeros(poly)
    for i in 2:num
        poly_acc = divrem(poly_acc * poly,f)[2]
    end
    return poly_acc
end

function Base.:string(poly::MyPolynomial{Int},var::Char)
    poly = remove_top_zeros(poly)
    coeffs = poly.coeffs
    i = 0
    s = string(first(coeffs))
    for i in 2:length(coeffs)
        s = string(s," + (",coeffs[i],var,"^",i-1,")")
    end
    return s
end

function derivative(poly::MyPolynomial{Int})
    coeffs = poly.coeffs
    p = poly.p
    new_coeffs = []

    for i in 2:length(coeffs)
        push!(new_coeffs,(i-1)*coeffs[i])
    end
    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end


function evaluate(poly::MyPolynomial{Int},x::Int)
    coeffs = poly.coeffs
    p = poly.p
    val = 0
    for i in 1:length(coeffs)
        val += coeffs[i] * x^(i-1)
    end
    return mod(val,p)
end

function Base.:string(poly::MyPolynomial{MyPolynomial{Int}},var::Char)
    poly = remove_top_zeros(poly)
    coeffs = poly.coeffs
    i = 0
    s = string(first(coeffs),var)
    for i in 2:length(coeffs)
        s = string(s," + [",string(coeffs[i],var),"]",var,"^",i-1)
    end
    return s
end

function Base.:print(poly::MyPolynomial,var::Char)
    println(string(poly,var))
end

a_int = ConstructMyPolynomial([1,1,1],3)
b_int = ConstructMyPolynomial([1,2,0],3)
c_int = ConstructMyPolynomial([1,2,2],3)

a_poly = ConstructMyPolynomial([a_int,b_int],3)
b_poly = ConstructMyPolynomial([a_int,c_int],3)
c_poly = ConstructMyPolynomial([c_int],3)
f = ConstructMyPolynomial([1,1,0,0,1],2)

powermod(ConstructMyPolynomial([0,1],2),9,f)

mod(ConstructMyPolynomial([0,1,0,1],2),f)

MyPolynomial{Int64}([0, 1, 0, 1], 3, 2)

## 3.1 RS Codes on $\mathbb{F}_p$ (50%)

### 3.1.1 RS Codes on $\mathbb{F}_{17}$

1. In $\mathbb{F}_{17}$, find a primitive element $\alpha$. 
2. Let $\mathcal{C} \subset \mathbb{F}_{17}^{16}$ be an RS code with parameters $[16,k,7]$. What is the value of $k$? 
3. Design and implement an encoding function `RSNum_Enc` that encodes a message $\boldsymbol{m} \in \mathbb{F}_{17}^k$ to a codeword $\boldsymbol{c} \in \mathcal{C}$. 
4. Let $\boldsymbol{y} = \boldsymbol{c} + \boldsymbol{e}$ be the received word where the channel introduces $t = \lfloor \frac{d-1}{2}\rfloor$ errors in random places. Design and implement a function `RSNum_Channel` for this. Julia function `StatsBase > Sample` can be helpful in randomly chose error locations.
5. Design and implement a function `RSNum_Syndrome` to calculate the syndrome vector and the syndrome polynomial from the received word $\boldsymbol{y}$. 
6. Design and implement a function `RSNum_KeyPolys` to find error locator polynomial $L(z)$ and error evaluator polynomial $E(z)$ from the syndrome polynomial $S(z)$. 
7. Design and implement a function `RSNum_Err` to find the estimated error vector $\hat{\boldsymbol{e}}$ and the estimated codeword $\hat{\boldsymbol{c}}$.
8. Design and implement a function `RSNum_Message` to find the estimated message $\hat{\boldsymbol{m}}$. 
9. Implement a function `RSNum_Dec` which uses the functions in Steps 5-8 as sub-routines for decoding.  

Provide necessary documentation.

#### 1. Find a Primitive Element $\alpha$

We use the function `find_primitve(p)`, which returns an array of all primitive elements in the finite field. We then pick one at random to be our $\alpha$.

In [4]:
function is_primitive_element(element,p)
    if modExpo(p,element,p-1) != 1
        return false
    end

    # Using ord(β) | p-1 for Fₚ.
    # Using p instead of q as p = q for integer ring finite fields.
    # We only need to check powers that are factors of p-1.
    factors = []
    for i in 2:isqrt(p-1)
        quot,rem = divrem(p-1,i)
        if rem == 0
            append!(factors,[i,quot])
        end
    end
    
    for t in factors
        if modExpo(p,element,t) == 1
            return false
        end
    end
    return true
end

# Find primitive elements for finite field Fₚ
function find_primitive(p)
    field = collect(0:p-1)
    prim_els = []
    for a in field
        if is_primitive_element(a,p)
            push!(prim_els,a)
        end
    end
    return prim_els
end

find_primitive(17)

8-element Vector{Any}:
  3
  5
  6
  7
 10
 11
 12
 14

In [5]:
# Define the prime number for our Finite Field
p = 17

# Get all primitive elements in that Finite Field
prim_els = find_primitive(p)

# Pick a primitive element at random to be α
α = prim_els[rand(1:length(prim_els))]
@show α

α = 12


12

#### 2. What is the value of $k$?

RS Codes are defined by three parameters: **$[n,k,d]$**, where $d = n - k + 1$.

Taking the RS Code given in the question:  **$[16,k,7]$**, we can see that $n = 16$ and $n - k + 1 = 7$. 

We can substitute in $n = 16$ to get: $16 - k + 1 = 7$. Solving this equation yields **$k = 10$**.

In [6]:
# Define the values of [n,k,d]
n = 16
k = 10
d = 7

7

#### 3. RS Encoding

In [7]:
# Helper Functions for RS Encoding

"""
generator_matrix(p,α,n,k)

Args:
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message

Returns:
A k×n matrix of type Matrix{Int} which is the generator matrix of the RS Code [n,k,d]
"""
function generator_matrix(p,α,n,k)
    # First row of the matrix will always be n 1s, so generate that
    gen_mat = ones(Int,1,n)
    
    for i in 1:k-1
        # Calculate the defining element for that row (αⁱ, where i goes up to k)
        αⁱ = modExpo(p,α,i)
        row = fill(αⁱ,1,n)
        for j in 1:n
            # Mulitply the defining element for the given row by ascending powers up to n
            row[j] = modExpo(p,row[j],j-1)
        end
        # Add the new row to the bottom of the matrix
        gen_mat = vcat(gen_mat,row)
    end
    return gen_mat
end


generator_matrix

In [8]:
"""
RSNum_Enc(message,p,α,n,k)

Args:
message: Matrix{Int} of dimension 1×k containing the message
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message

Returns:
A 1×n matrix of type Matrix{Int}, which is the encoded message
"""

function RSNum_Enc(message,p,α,n,k)
    if typeof(message) != Matrix{Int}
        println("Please enter message in the format of a 1×k matrix")
        println("Type of message should be Matrix{Int}, not $(typeof(message))")
        return -1
    end
    if length(message) != k
        println("Length of message should be $k, which is the value of k provided")
        return -1
    end

    # Ensure each element in our message is in Fₚ
    # If not then, perform modulo to make it comply
    message = [mod(i,p) for i in message]

    # Calculate the Generator Matrix for the given parameters
    G = generator_matrix(p,α,n,k)

    # Calculate the code by multiplying the message and the generator matrix
    code = message * G
    code = [mod(i,p) for i in code]

    return code

end

RSNum_Enc (generic function with 1 method)

In [9]:
# Test: Encode a message
message = [7 3 8 7 1 9 4 4 0 0]
codeword = RSNum_Enc(message,p,α,n,k)

1×16 Matrix{Int64}:
 9  5  16  8  9  14  0  16  14  9  15  8  0  1  10  12

#### 4. Simulating the Channel

In [10]:
"""
RSNum_Channel(code,p,n,d)

Args:
code: Matrix{Int} of dimension 1×n, the codeword to be transmitted accross the channel
p: The prime number that defines the Finite Field Fₚ
n: The length of the code
d: d as defined for the RS Code

Returns:
A 1×n matrix of type Matrix{Int}, which is the codeword with error introduced
"""

function RSNum_Channel(code,p,n,d)
    # Maximum number of errors
    t::Int = floor((d-1)/2)

    # Randomly choose t unique locations of the errors
    I = sample((1:n),t,replace=false)
    @show I
    # Build error vector, which has non-zero elements at eᵢ where i∈I
    e = []
    for i in 1:n
        if i in I
            push!(e,rand(1:p))
        else
            push!(e,0)
        end
    end

    # Convert e from type Vector{Int} to Matrix{Int}
    # This makes it the same type and shape as the code, allowing for addition
    e = reshape(e,1,n)
    @show e
    # Add error to the code
    changed_code = code + e
    changed_code = [mod(i,p) for i in changed_code]

    return changed_code
end


RSNum_Channel (generic function with 1 method)

In [11]:
# Test: Transmit codeword through channel
received = RSNum_Channel(codeword,p,n,d)

I = [5, 13, 9]
e = Any[0 0 0 0 9 0 0 0 5 0 0 0 10 0 0 0]


1×16 Matrix{Int64}:
 9  5  16  8  1  14  0  16  2  9  15  8  10  1  10  12

#### 5. Syndrome Vector and Polynomial

In [12]:
# Helper functions

"""
parity_check_matrix(p,α,n,k)

Args:
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message

Returns:
A (n-k)×n matrix of type Matrix{Int} which is the parity-check matrix of the RS Code [n,k,d]

"""

function parity_check_matrix(p,α,n,k)
    pc_mat = zeros(Int,n-k,n)
    for i in 1:n-k
        # Calculate the defining element for that row (αⁱ, where i goes up to k)
        αⁱ = modExpo(p,α,i)
        row = fill(αⁱ,1,n)
        for j in 1:n
            # Multiply the defining element for the given row by ascending powers up to n
            pc_mat[i+(j-1)*(n-k)] = modExpo(p,row[j],j-1)
        end
    end
    return pc_mat
end

parity_check_matrix(p,α,n,k)


6×16 Matrix{Int64}:
 1  12   8  11  13   3   2   7  16   5   9   6   4  14  15  10
 1   8  13   2  16   9   4  15   1   8  13   2  16   9   4  15
 1  11   2   5   4  10   8   3  16   6  15  12  13   7   9  14
 1  13  16   4   1  13  16   4   1  13  16   4   1  13  16   4
 1   3   9  10  13   5  15  11  16  14   8   7   4  12   2   6
 1   2   4   8  16  15  13   9   1   2   4   8  16  15  13   9

In [13]:
# Verifying that the identity GHᵀ = 0 holds
# Done to test generator_matrix and parity_check_matrix functions
G = generator_matrix(p,α,n,k)
H = parity_check_matrix(p,α,n,k)

A = G * transpose(H)
A = [mod(i,p) for i in A]

C = codeword * transpose(H)
C = [mod(i,p) for i in C]


1×6 Matrix{Int64}:
 0  0  0  0  0  0

In [14]:
function RSNum_Syndrome(changed_code,p,α,n,k)
    # Calculate the parity-check Matrix
    H = parity_check_matrix(p,α,n,k)

    # Calculate syndrome vector
    syndrome_vector = changed_code * transpose(H)
    syndrome_vector = [mod(i,p) for i in syndrome_vector]

    # Return the syndrome vector and syndrome polynomial
    return (syndrome_vector,ConstructMyPolynomial(vec(syndrome_vector),p))
end

RSNum_Syndrome (generic function with 1 method)

In [15]:
# Test: Calculate Syndrome Vector/Polynomial from Received Codeword
s_vector,s_poly = RSNum_Syndrome(received,p,α,n,k)
println("Syndrome Vector: ", s_vector)
println("Syndrome Polynomial S(z) = ", string(s_poly,'z'))

Syndrome Vector: [16 3 8 7 16 3]
Syndrome Polynomial S(z) = 16 + (3z^1) + (8z^2) + (7z^3) + (16z^4) + (3z^5)


#### 6. Error Locator Polynomial and Error Evaluator Polynomial

In [16]:
# Helper functions

function partial_euclidean_algorithm(a::MyPolynomial,b::MyPolynomial,stop::Int)
    p = a.p
    a = remove_top_zeros(a)
    b = remove_top_zeros(b)

    x0,x1 = ConstructMyPolynomial([1],p),ConstructMyPolynomial([0],p)
    y0,y1 = ConstructMyPolynomial([0],p),ConstructMyPolynomial([1],p)

    if b == ConstructMyPolynomial([0],p)
        return (a,x0,y0)
    end

    r = b

    last_run = false

     while r.order > stop || !last_run
        if r.order <= stop
            last_run = true
        end
        q,r = divrem(a,b)
        #@show q,r
        a = b
        b = r

        x0, x1 = x1, x0 - q*x1
        y0, y1 = y1, y0 - q*y1
    end
    return (a,x0,y0)
end
    
t = Int(floor((d+1)/2))
poly_b = ConstructMyPolynomial(append!(zeros(Int,d-1),[1]),p)

partial_euclidean_algorithm(poly_b,s_poly,t-1)


(MyPolynomial{Int64}([11, 4, 8, 9], 3, 17), MyPolynomial{Int64}([6, 9], 1, 17), MyPolynomial{Int64}([6, 14, 14], 2, 17))

In [17]:
function RSNum_KeyPolys(s_poly,p,α,n,d)
    t::Int = floor((d-1)/2)
    z = ConstructMyPolynomial(append!(zeros(Int,d-1),[1]),p)
    r,b,a = partial_euclidean_algorithm(z,s_poly,t-1)

    E_prime = divrem(r,z)[2]
    L_prime = divrem(a,z)[2]

    c_inv = modulo_div(1,last(L_prime.coeffs),p)

    L = L_prime * c_inv
    E = E_prime * c_inv

    return L,E
end


RSNum_KeyPolys (generic function with 1 method)

In [18]:
L,E = RSNum_KeyPolys(s_poly,p,α,n,d)

(MyPolynomial{Int64}([1, 1, 1, 1], 3, 17), MyPolynomial{Int64}([16, 2, 10], 2, 17))

#### 7. Estimating Error

In [19]:
# Helper Functions

function error_locations(L,p,α,n)
    I = []
    for k in 0:n-1
        if evaluate(L,modulo_div(1,α^k,p)) == 0
            push!(I,k)
        end
    end
    return I
end

function error_values(E,I,p,α)
    dL = derivative(L)
    ê = []
    for k in 0:n-1
        if k in I
            α⁻ᵏ = modulo_div(1,α^k,p)
            E_of_α⁻ᵏ = evaluate(E,α⁻ᵏ)
            dL_of_α⁻ᵏ = evaluate(dL,α⁻ᵏ)
            eₖ = mod(-1 *  modulo_div(E_of_α⁻ᵏ, dL_of_α⁻ᵏ,p),p)
            push!(ê,eₖ)
        else
            push!(ê,0)
        end
    end
    return reshape(ê,1,n)
end


error_values (generic function with 1 method)

In [20]:
function RSNum_Err(L,E,p,α,n,received)
    I = error_locations(L,p,α,n)
    ê = error_values(E,I,p,α)
    ĉ = [mod(i,p) for i in received - ê]

    return ê,ĉ
end

RSNum_Err (generic function with 1 method)

In [21]:
ê,ĉ = RSNum_Err(L,E,p,α,n,received)
@show ê,ĉ

(ê, ĉ) = (Any[0 0 0 0 9 0 0 0 5 0 0 0 10 0 0 0], [9 5 16 8 9 14 0 16 14 9 15 8 0 1 10 12])


(Any[0 0 … 0 0], [9 5 … 10 12])

#### RS Decoding

In [22]:
k = 10
p = 17
G_prime = []
for i in 1:(k^2)
    push!(G_prime,G[i])
end
G_prime = reshape(G_prime,k,k)
G_prime = [ConstructMyPolynomial([i],p) for i in G_prime]

#inv(G_prime)

10×10 Matrix{MyPolynomial{Int64}}:
 MyPolynomial{Int64}([1], 0, 17)  …  MyPolynomial{Int64}([1], 0, 17)
 MyPolynomial{Int64}([1], 0, 17)     MyPolynomial{Int64}([5], 0, 17)
 MyPolynomial{Int64}([1], 0, 17)     MyPolynomial{Int64}([8], 0, 17)
 MyPolynomial{Int64}([1], 0, 17)     MyPolynomial{Int64}([6], 0, 17)
 MyPolynomial{Int64}([1], 0, 17)     MyPolynomial{Int64}([13], 0, 17)
 MyPolynomial{Int64}([1], 0, 17)  …  MyPolynomial{Int64}([14], 0, 17)
 MyPolynomial{Int64}([1], 0, 17)     MyPolynomial{Int64}([2], 0, 17)
 MyPolynomial{Int64}([1], 0, 17)     MyPolynomial{Int64}([10], 0, 17)
 MyPolynomial{Int64}([1], 0, 17)     MyPolynomial{Int64}([16], 0, 17)
 MyPolynomial{Int64}([1], 0, 17)     MyPolynomial{Int64}([12], 0, 17)

In [23]:
# function decode_corrected_code(ĉ,p,α,n,k)
#     # Calculate Generator Matrix
#     G = generator_matrix(p,α,n,k)
#     G_prime = []
#     for i in 1:(k^2)
#         push!(G_prime,G[i])
#     end
#     G_prime = reshape(G_prime,k,k)
#     G_prime = [ConstructMyPolynomial([i],p) for i in G_prime]
#     code_prime = [ĉ[i] for i in 1:k]

#     return code_prime * inv(G_prime)
# end

# decode_corrected_code(ĉ,p,α,n,k)

    

### 3.1.2 RS Codes on $\mathbb{F}_{p}$

The above code should work well for other choices of $p$, $\alpha$, $[n,k,d]$. Try another $p$ with $30<p<100$ and demonstrate the correctness of your code. 

## 3.2 RS Codes on $\mathbb{F}_{p^k}$ (50%)

### 3.2.1 RS Codes on $\mathbb{F}_{16}$

Now we consider the finite field $\mathbb{F}_{16} = \mathbb{F}_2[z]/z^4 + z +1$, and an RS code $\mathcal{C} \subset \mathbb{F}_{16}$ with parameters $[15,k,7]$. 

Repeat the steps in 3.1.1 and change the function names from `RSNum_...` to `RSPoly_...`. Provide necessary documentation. 

#### 1. Finding a primitive element $\alpha$

In [24]:
function is_primitive_element(element,poly)
    p = poly.p
    poly = remove_top_zeros(poly)
    element = remove_top_zeros(element)
    q = p^poly.order # size of finite field q = pᵈ

    element_acc = ConstructMyPolynomial(element.coeffs,element.p)
    for t in 2:q
        element_acc = element_acc * element
        if divrem(element_acc,poly)[2] == ConstructMyPolynomial([1],p)
            if t == q - 1
                return true
            else
                return false
            end
        end
    end
end

function find_primitive(poly)
    poly = remove_top_zeros(poly)
    p = poly.p
    q = p^poly.order
    prim_els = []

    for i in 1:q-2
        el = divrem(ConstructMyPolynomial(append!(zeros(Int,i),[1]),p),poly)[2]
        #print(el,'x')
        if is_primitive_element(el,poly)
            push!(prim_els,el)
        end
    end

    return prim_els
end

find_primitive(ConstructMyPolynomial([1,1,0,0,1],2))

8-element Vector{Any}:
 MyPolynomial{Int64}([0, 1], 1, 2)
 MyPolynomial{Int64}([0, 0, 1], 2, 2)
 MyPolynomial{Int64}([1, 1], 1, 2)
 MyPolynomial{Int64}([1, 1, 0, 1], 3, 2)
 MyPolynomial{Int64}([1, 0, 1], 2, 2)
 MyPolynomial{Int64}([0, 1, 1, 1], 3, 2)
 MyPolynomial{Int64}([1, 0, 1, 1], 3, 2)
 MyPolynomial{Int64}([1, 0, 0, 1], 3, 2)

In [25]:
# Define the Finite Field
p = 2
f = ConstructMyPolynomial([1,1,0,0,1],p)

# Get all primitive elements in that Finite Field
prim_els = find_primitive(f)

# Pick a primitive element at random to be α
α = prim_els[rand(1:length(prim_els))]
print(α,'x')

1 + (0x^1) + (0x^2) + (1x^3)


#### 2. What is the value of $k$?

RS Codes are defined by three parameters: **$[n,k,d]$**, where $d = n - k + 1$.

Taking the RS Code given in the question:  **$[16,k,7]$**, we can see that $n = 15$ and $n - k + 1 = 7$. 

We can substitute in $n = 15$ to get: $15 - k + 1 = 7$. Solving this equation yields **$k = 9$**.

In [26]:
# Define the values of [n,k,d]
n = 15
k = 9
d = 7

7

#### 3. RS Encoding

In [27]:
# Helper Functions for RS Encoding

"""
generator_matrix_polu(p,f,α,n,k)

Args:
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
α: The chosen primitive element from Fq
n: The length of the code
k: The length of the message

Returns:
A k×n matrix of type Matrix{MyPolynomial{Int}} which is the generator matrix of the RS Code [n,k,d]
"""
function generator_matrix_poly(p,f,α,n,k)
    # First row of the matrix will always be n 1s, so generate that
    gen_mat = reshape(ones_poly(n,p),1,n)
    
    for i in 1:k-1
        # Calculate the defining element for that row (αⁱ, where i goes up to k)
        αⁱ = powermod(α,i,f)
        row = reshape(fill(αⁱ,n),1,n)
        for j in 1:n
            # Mulitply the defining element for the given row by ascending powers up to n
            row[j] = powermod(row[j],j-1,f)
        end
        # Add the new row to the bottom of the matrix
        gen_mat = vcat(gen_mat,row)
    end
    return gen_mat
end

generator_matrix_poly(2,f,α,n,k)


9×15 Matrix{Any}:
 MyPolynomial{Int64}([1], 0, 2)  …  MyPolynomial{Int64}([1], 0, 2)
 MyPolynomial{Int64}([1], 0, 2)     MyPolynomial{Int64}([0, 1], 1, 2)
 MyPolynomial{Int64}([1], 0, 2)     MyPolynomial{Int64}([0, 0, 1], 2, 2)
 MyPolynomial{Int64}([1], 0, 2)     MyPolynomial{Int64}([0, 0, 0, 1], 3, 2)
 MyPolynomial{Int64}([1], 0, 2)     MyPolynomial{Int64}([1, 1], 1, 2)
 MyPolynomial{Int64}([1], 0, 2)  …  MyPolynomial{Int64}([0, 1, 1], 2, 2)
 MyPolynomial{Int64}([1], 0, 2)     MyPolynomial{Int64}([0, 0, 1, 1], 3, 2)
 MyPolynomial{Int64}([1], 0, 2)     MyPolynomial{Int64}([1, 1, 0, 1], 3, 2)
 MyPolynomial{Int64}([1], 0, 2)     MyPolynomial{Int64}([1, 0, 1], 2, 2)

In [28]:
"""
RSPoly_Enc(message,p,f,α,n,k)

Args:
message: Matrix{MyPolynomial{Int}} of dimension 1×k containing the message
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
α: The chosen primitive element from Fq
n: The length of the code
k: The length of the message

Returns:
A 1×n matrix of type Matrix{MyPolynomial{Int}}, which is the encoded message
"""

function RSPoly_Enc(message,p,f,α,n,k)
    if typeof(message) != Matrix{MyPolynomial{Int}}
        println("Please enter message in the format of a 1×k matrix")
        println("Type of message should be Matrix{MyPolynomial{Int}}, not $(typeof(message))")
        return -1
    end
    if length(message) != k
        println("Length of message should be $k, which is the value of k provided")
        return -1
    end

    # Ensure each element in our message is in Fq
    # If not then, perform modulo to make it comply
    message = [mod(i,f) for i in message]

    # Calculate the Generator Matrix for the given parameters
    G = generator_matrix_poly(p,f,α,n,k)

    # Calculate the code by multiplying the message and the generator matrix
    code = message * G
    code = [mod(i,f) for i in code]

    return code

end

RSPoly_Enc (generic function with 1 method)

In [29]:
# Test: Encode a message

message = [
            ConstructMyPolynomial([0,1],2),
            ConstructMyPolynomial([0,0,1],2),
            ConstructMyPolynomial([0,0,0,1],2),
            ConstructMyPolynomial([0,1,1],2),
            ConstructMyPolynomial([0,0,1,1],2),
            ConstructMyPolynomial([1,0,1],2),
            ConstructMyPolynomial([1,0,0,1],2),
            ConstructMyPolynomial([1,0,1,1],2),
            ConstructMyPolynomial([1,1,1,1],2)
            ]

message = reshape(message,1,k)

codeword = RSPoly_Enc(message,p,f,α,n,k)

1×15 Matrix{MyPolynomial{Int64}}:
 MyPolynomial{Int64}([0, 1, 0, 1], 3, 2)  …  MyPolynomial{Int64}([1, 0, 1], 2, 2)

#### 4. Simulating the Channel

In [30]:
# Helper Functions

function finite_field_elements(p,f::MyPolynomial)
    q = p^f.order
    elements = [ConstructMyPolynomial([0],2),ConstructMyPolynomial([1],2)]
    for i in 1:q-2
        el = divrem(ConstructMyPolynomial(append!(zeros(Int,i),[1]),p),f)[2]
        push!(elements,el)
    end
    return elements
end

finite_field_elements(2,ConstructMyPolynomial([1,1,0,0,1],2))

16-element Vector{MyPolynomial{Int64}}:
 MyPolynomial{Int64}([0], 0, 2)
 MyPolynomial{Int64}([1], 0, 2)
 MyPolynomial{Int64}([0, 1], 1, 2)
 MyPolynomial{Int64}([0, 0, 1], 2, 2)
 MyPolynomial{Int64}([0, 0, 0, 1], 3, 2)
 MyPolynomial{Int64}([1, 1], 1, 2)
 MyPolynomial{Int64}([0, 1, 1], 2, 2)
 MyPolynomial{Int64}([0, 0, 1, 1], 3, 2)
 MyPolynomial{Int64}([1, 1, 0, 1], 3, 2)
 MyPolynomial{Int64}([1, 0, 1], 2, 2)
 MyPolynomial{Int64}([0, 1, 0, 1], 3, 2)
 MyPolynomial{Int64}([1, 1, 1], 2, 2)
 MyPolynomial{Int64}([0, 1, 1, 1], 3, 2)
 MyPolynomial{Int64}([1, 1, 1, 1], 3, 2)
 MyPolynomial{Int64}([1, 0, 1, 1], 3, 2)
 MyPolynomial{Int64}([1, 0, 0, 1], 3, 2)

In [31]:
"""
RSPoly_Channel(code,p,f,n,d)

Args:
code: Matrix{Int} of dimension 1×n, the codeword to be transmitted accross the channel
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
n: The length of the code
d: d as defined for the RS Code

Returns:
A 1×n matrix of type Matrix{MyPolynomial{Int}}, which is the codeword with error introduced
"""

function RSPoly_Channel(code,p,f,n,d)

    # q = pᵈ
    q = p^f.order

    # Maximum number of errors
    t::Int = floor((d-1)/2)

    # Find all elements in the Finite Field
    ff_els = finite_field_elements(p,f)

    # Randomly choose t unique locations of the errors
    I = sample((1:n),t,replace=false)
    @show I
    # Build error vector, which has non-zero elements at eᵢ where i∈I
    e = []
    for i in 1:n
        if i in I
            push!(e,ff_els[rand(1:q)])
        else
            push!(e,ConstructMyPolynomial([0],2))
        end
    end

    # Convert e from type Vector{MyPolynomial} to Matrix{MyPolynomial}
    # This makes it the same type and shape as the code, allowing for addition
    e = reshape(e,1,n)
    @show e
    # Add error to the code
    changed_code = code + e
    changed_code = [mod(i,p) for i in changed_code]

    return changed_code
end


RSPoly_Channel (generic function with 1 method)

In [32]:
# Test: Transmit codeword through channel
received = RSPoly_Channel(codeword,p,f,n,d)

I = [3, 11, 7]
e = Any[MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([0, 0, 0, 1], 3, 2) MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([0, 0, 1], 2, 2) MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([0, 1, 1, 1], 3, 2) MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([0], 0, 2)]


1×15 Matrix{MyPolynomial{Int64}}:
 MyPolynomial{Int64}([0, 1, 0, 1], 3, 2)  …  MyPolynomial{Int64}([1, 0, 1], 2, 2)

#### 5. Syndrome Vector and Polynomial

In [55]:
# Helper functions

"""
parity_check_matrix_poly(p,f,α,n,k)

Args:
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
α: The chosen primitive element from Fq
n: The length of the code
k: The length of the message

Returns:
A (n-k)×n matrix of type Matrix{MyPolynomial} which is the parity-check matrix of the RS Code [n,k,d]

"""

function parity_check_matrix_poly(p,f,α,n,k)
    pc_mat::Matrix{MyPolynomial} = reshape(zeros_poly((n-k)*n,p),n-k,n)
    for i in 1:n-k
        # Calculate the defining element for that row (αⁱ, where i goes up to k)
        αⁱ = powermod(α,i,f)
        row = reshape(fill(αⁱ,n),1,n)
        for j in 1:n
            # Multiply the defining element for the given row by ascending powers up to n
            pc_mat[i+(j-1)*(n-k)] = powermod(row[j],j-1,f)
        end
    end
    return pc_mat
end

H =parity_check_matrix_poly(p,f,α,n,k)


6×15 Matrix{MyPolynomial}:
 MyPolynomial{Int64}([1], 0, 2)  …  MyPolynomial{Int64}([0, 1], 1, 2)
 MyPolynomial{Int64}([1], 0, 2)     MyPolynomial{Int64}([0, 0, 1], 2, 2)
 MyPolynomial{Int64}([1], 0, 2)     MyPolynomial{Int64}([0, 0, 0, 1], 3, 2)
 MyPolynomial{Int64}([1], 0, 2)     MyPolynomial{Int64}([1, 1], 1, 2)
 MyPolynomial{Int64}([1], 0, 2)     MyPolynomial{Int64}([0, 1, 1], 2, 2)
 MyPolynomial{Int64}([1], 0, 2)  …  MyPolynomial{Int64}([0, 0, 1, 1], 3, 2)

In [68]:
function RSPoly_Syndrome(changed_code,p,f,α,n,k)
    # Calculate the parity-check Matrix
    H = parity_check_matrix_poly(p,f,α,n,k)

    # Calculate syndrome vector
    syndrome_vector = changed_code * permutedims(H)
    syndrome_vector = [mod(i,f) for i in syndrome_vector]

    # Return the syndrome vector and syndrome polynomial
    return (syndrome_vector,ConstructMyPolynomial(vec(syndrome_vector),p))
end

RSPoly_Syndrome (generic function with 1 method)

In [71]:
# Test: Calculate Syndrome Vector/Polynomial from Received Codeword
s_vector,s_poly = RSPoly_Syndrome(received,p,f,α,n,k)
println("Syndrome Vector: ", s_vector)
println("Syndrome Polynomial S(z) = ", string(s_poly,'z'))

Syndrome Vector: MyPolynomial{Int64}[MyPolynomial{Int64}([0, 1, 1, 1], 3, 2) MyPolynomial{Int64}([1, 1], 1, 2) MyPolynomial{Int64}([0, 0, 0, 1], 3, 2) MyPolynomial{Int64}([0], 0, 2) MyPolynomial{Int64}([1, 0, 1, 1], 3, 2) MyPolynomial{Int64}([0, 0, 1, 1], 3, 2)]
Syndrome Polynomial S(z) = 0 + (1z^1) + (1z^2) + (1z^3) + [1 + (1z^1)]z^1 + [0 + (0z^1) + (0z^2) + (1z^3)]z^2 + [0]z^3 + [1 + (0z^1) + (1z^2) + (1z^3)]z^4 + [0 + (0z^1) + (1z^2) + (1z^3)]z^5


#### 6. Error Locator Polynomial and Error Evaluator Polynomial

In [86]:
# Helper functions

function partial_euclidean_algorithm(a::MyPolynomial,b::MyPolynomial,stop::Int)
    p = a.p
    a = remove_top_zeros(a)
    b = remove_top_zeros(b)

    x0,x1 = ConstructMyPolynomial([1],p),ConstructMyPolynomial([0],p)
    y0,y1 = ConstructMyPolynomial([0],p),ConstructMyPolynomial([1],p)

    if b == ConstructMyPolynomial([0],p)
        return (a,x0,y0)
    end

    r = b

    last_run = false

     while r.order > stop || !last_run
        if r.order <= stop
            last_run = true
        end
        q,r = divrem(a,b)
        #@show q,r
        a = b
        b = r

        x0, x1 = x1, x0 - q*x1
        y0, y1 = y1, y0 - q*y1
    end
    return (a,x0,y0)
end

partial_euclidean_algorithm (generic function with 1 method)

In [84]:
function RSPoly_KeyPolys(s_poly,p,α,n,d)
    t::Int = floor((d-1)/2)
    z = ConstructMyPolynomial(append!(zeros_poly(d-1,p),[ConstructMyPolynomial([1],p)]),p)
    r,b,a = partial_euclidean_algorithm_poly(z,s_poly,t-1)

    E_prime = divrem(r,z)[2]
    L_prime = divrem(a,z)[2]

    L = divrem(L_prime, ConstructMyPolynomial(c_inv,p))
    E = divrem(E_prime, ConstructMyPolynomial(c_inv,p))

    return L,E
end

RSPoly_KeyPolys (generic function with 1 method)

In [87]:
L,E = RSPoly_KeyPolys(s_poly,p,α,n,d)

LoadError: MethodError: no method matching iterate(::MyPolynomial{Int64})
[0mClosest candidates are:
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m) at range.jl:664
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m, [91m::Int64[39m) at range.jl:664
[0m  iterate([91m::T[39m) where T<:Union{Base.KeySet{var"#s77", var"#s76"} where {var"#s77", var"#s76"<:Dict}, Base.ValueIterator{var"#s75"} where var"#s75"<:Dict} at dict.jl:693
[0m  ...

### 3.2.2 RS Codes on $\mathbb{F}_{2^k}$

The above code should work for other choices of $k$ in $\mathbb{F}_{2^k}$. In 3.2.1, we have worked on the case where $k=4$. Try a different $k$ with $5 \le k \le 10$. 

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.